In [1]:
from numpy.random import seed
seed(1)
import tensorflow as tf
tf.random.set_seed(2)

import os, argparse, sys
# sys.path.append('./')
ROOT_DIR = os.path.dirname(os.path.dirname(os.path.abspath('__file__')))
os.chdir(ROOT_DIR)
sys.path.append(ROOT_DIR)
# importing scripts in scripts folder
from scripts import config as src

**** scripts/config.py IMPORTED!!!
**** PROJECT FOLDER ,  /home/pgundogdu/projects/signalization_prior_knowledge_based_nn


In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import LeavePGroupsOut, train_test_split
from sklearn.metrics import homogeneity_score, completeness_score, v_measure_score, adjusted_rand_score, adjusted_mutual_info_score, fowlkes_mallows_score

epochs_default=100 # for model design
batch_default=10   # for model design
val_split=0.1      # for model design
rand_state = 91    # dataset split
test_size = 0.3    # train_test_split
kf_split = 5       # KFold

from numba import cuda
from tensorflow import keras


In [3]:
loc_output = 'models/NN/exper_mouse/leavePgroupsOut'
save_path = src.define_folder(loc_output)
save_path

FOLDER information,  models/NN/exper_mouse/leavePgroupsOut/


'models/NN/exper_mouse/leavePgroupsOut/'

In [4]:
df_learning = pd.read_csv('data/external/exper_mouse/1-3_integrated_NNtraining.txt'
                          , sep='\t'
                          , index_col=0).T.drop(columns='Weight')#.reset_index()
df_learning = df_learning.rename(columns={'Label':'cell_type'})
df_learning = df_learning[sorted(df_learning.columns[1:]) + [df_learning.columns[0] ]]
print(df_learning.shape)
df_learning.head()

(402, 9438)


Sample,a1bg,a1cf,a2m,aaas,aacs,aagab,aak1,aamp,aanat,aars,...,zswim7,zufsp,zw10,zwilch,zxdb,zxdc,zyx,zzef1,zzz3,cell_type
S1,0,0,0.0,103.853,0.0586557,0,3.97772,136.672,0,28.4994,...,0,0,0.278639,0,0.0,0.0,297.919,37.6855,0,BMDC
S2,0,0,0.0,0,0,0,0.00742256,440.725,0,3.15802,...,0,147.538,0,0,0.0,0.0,0.242888,0,0,BMDC
S3,0,0,0.0,0,0,0,0.0261848,118.676,0,0,...,0,17.7033,0.0863634,0,0.0,0.0,6.75353,0.132011,0.0176152,BMDC
S4,0,0,0.0,0,0,0,0,355.623,0,9.10864,...,0,0.537516,0.0636703,0,0.0,0.0,0.274689,9.72489,0,BMDC
S5,0,0,0.0,0,0,0,0,151.626,0,19.3458,...,0,128.928,0,0.204522,0.0,0.0,59.2561,44.4307,0,BMDC


In [5]:
scaler = MinMaxScaler()
df_learning.iloc[:, :-1] = scaler.fit_transform(df_learning.iloc[:, :-1])
print(df_learning.shape)
df_learning.head()

(402, 9438)


Sample,a1bg,a1cf,a2m,aaas,aacs,aagab,aak1,aamp,aanat,aars,...,zswim7,zufsp,zw10,zwilch,zxdb,zxdc,zyx,zzef1,zzz3,cell_type
S1,0,0,0,0.193008,0.000101553,0,0.0968245,0.10984,0,0.102434,...,0,0,0.00143474,0,0,0,0.413223,0.652245,0,BMDC
S2,0,0,0,0,0,0,0.000180678,0.354199,0,0.0113507,...,0,0.302217,0,0,0,0,0.000336894,0,0,BMDC
S3,0,0,0,0,0,0,0.000637382,0.0953768,0,0,...,0,0.0362633,0.000444694,0,0,0,0.00936737,0.00228479,9.9952e-05,BMDC
S4,0,0,0,0,0,0,0,0.285804,0,0.0327387,...,0,0.00110105,0.000327845,0,0,0,0.000381003,0.168314,0,BMDC
S5,0,0,0,0,0,0,0,0.121858,0,0.0695334,...,0,0.264095,0,0.000588339,0,0,0.0821901,0.768988,0,BMDC


In [6]:
df_scaled = df_learning.reset_index(drop=True)
print(df_scaled.shape)
df_scaled.head()

(402, 9438)


Sample,a1bg,a1cf,a2m,aaas,aacs,aagab,aak1,aamp,aanat,aars,...,zswim7,zufsp,zw10,zwilch,zxdb,zxdc,zyx,zzef1,zzz3,cell_type
0,0,0,0,0.193008,0.000101553,0,0.0968245,0.10984,0,0.102434,...,0,0,0.00143474,0,0,0,0.413223,0.652245,0,BMDC
1,0,0,0,0,0,0,0.000180678,0.354199,0,0.0113507,...,0,0.302217,0,0,0,0,0.000336894,0,0,BMDC
2,0,0,0,0,0,0,0.000637382,0.0953768,0,0,...,0,0.0362633,0.000444694,0,0,0,0.00936737,0.00228479,9.9952e-05,BMDC
3,0,0,0,0,0,0,0,0.285804,0,0.0327387,...,0,0.00110105,0.000327845,0,0,0,0.000381003,0.168314,0,BMDC
4,0,0,0,0,0,0,0,0.121858,0,0.0695334,...,0,0.264095,0,0.000588339,0,0,0.0821901,0.768988,0,BMDC


In [7]:
df_pathway_signaling = pd.read_csv('data/processed/pbk_layer_mmu.txt', index_col=0)
df_pathway_metsig = pd.read_csv('data/processed/pbk_layer_mmu_metsig.txt', index_col=0)
df_ppi = pd.read_csv('data/processed/pbk_layer_ppi.txt', index_col=0)
df_ppitf = pd.read_csv('data/processed/pbk_layer_ppitf.txt', index_col=0)

print(df_pathway_signaling.shape)
print(df_pathway_metsig.shape)
print(df_ppi.shape)
print(df_ppitf.shape)

df_scaled = pd.concat([df_scaled.iloc[:, :-1].astype(np.float32).reset_index(drop=True), df_scaled.iloc[:, -1:].reset_index(drop=True)], axis=1)
print(df_scaled.info())
df_scaled_with_pathway_signaling = df_scaled.iloc[:, df_scaled.columns.isin( sorted(list(df_pathway_signaling.index) )+ ['cell_type'] )]
df_scaled_with_pathway_metsig = df_scaled.iloc[:, df_scaled.columns.isin( sorted(list(df_pathway_metsig.index) )+ ['cell_type'] )]
df_scaled_with_ppi = df_scaled.iloc[:, df_scaled.columns.isin( sorted(list(df_ppi.index) )+ ['cell_type'] )]
df_scaled_with_ppitf = df_scaled.iloc[:, df_scaled.columns.isin( sorted(list(df_ppitf.index) )+ ['cell_type'] )]

print(df_scaled_with_pathway_signaling.shape)
print(df_scaled_with_pathway_metsig.shape)
print(df_scaled_with_ppi.shape)
print(df_scaled_with_ppitf.shape)

(3737, 92)
(3922, 250)
(3553, 348)
(9437, 696)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 402 entries, 0 to 401
Columns: 9438 entries, a1bg to cell_type
dtypes: float32(9437), object(1)
memory usage: 14.5+ MB
None
(402, 1646)
(402, 3923)
(402, 3554)
(402, 9438)


In [8]:
callbacks = [keras.callbacks.EarlyStopping(monitor="val_loss" # Stop training when `val_loss` is no longer improving
                                       , min_delta=1e-5   # "no longer improving" being defined as "no better than 1e-5 less"
                                       , patience=3       # "no longer improving" being further defined as "for at least 3 epochs"
                                       , verbose=1 ) ]

In [9]:
n_groups_ = [2,4,6,8]
ohe = OneHotEncoder()
# n_groups_ = [4]

X = df_scaled.iloc[:, :-1].values
y = df_scaled.iloc[:, -1:].values
y_ohe = ohe.fit_transform(y).toarray()
groups = df_scaled.iloc[:, -1].values


In [47]:
# FULLY CONNECTED 1-LAYER-100
# df_scaler_use = df_scaled.copy()
# df_bio_use = None
# second_layer=False
# size_units = 100
# design_type='fully'
# design = 'full_gene_space_1_layer_dense100'

# FULLY CONNECTED 1-LAYER-796
# df_scaler_use = df_scaled.copy()
# df_bio_use = None
# second_layer=False
# size_units = 796
# design_type='fully'
# design = 'full_gene_space_1_layer_dense796'

In [55]:
# # FULLY CONNECTED 1-LAYER-100 with signaling gene space
# df_scaler_use = df_scaled_with_pathway_signaling.copy()
# df_bio_use = None
# second_layer=False
# size_units = 100
# design_type='fully'
# design = 'signaling_gene_space_1_layer_dense100'

# # # FULLY CONNECTED 1-LAYER-796 with signaling gene space
# df_scaler_use = df_scaled_with_pathway_signaling.copy()
# df_bio_use = None
# second_layer=False
# size_units = 796
# design_type='fully'
# design = 'signaling_gene_space_1_layer_dense796'

In [62]:
# # FULLY CONNECTED 1-LAYER-100 with metabolic signaling gene space
# df_scaler_use = df_scaled_with_pathway_metsig.copy()
# df_bio_use = None
# second_layer=False
# size_units = 100
# design_type='fully'
# design = 'metsig_gene_space_1_layer_dense100'

# # FULLY CONNECTED 1-LAYER-796 with metabolic signaling gene space
# df_scaler_use = df_scaled_with_pathway_metsig.copy()
# df_bio_use = None
# second_layer=False
# size_units = 796
# design_type='fully'
# design = 'metsig_gene_space_1_layer_dense796'

In [ ]:
f dense_nodes != 0:
        df_dense = pd.DataFrame(df_org.columns[:-1]).set_index('Sample')
        for i in range(dense_nodes):
            df_dense['dense'+str(i)] = 1.0
        df_first_hidden_layer = df_dense.copy()
    print(df_first_hidden_layer)
    if ( ( bio_knowledge != 'None' ) and ( dense_nodes != 0 ) ):
        df_first_hidden_layer = pd.merge(df_dense, df_bio, left_index=True, right_index=True, how='left').fillna(0)
        

In [10]:
# # SIGNALING 1-LAYER
df_scaler_use = df_scaled_with_pathway_signaling.copy()
df_bio_use = df_pathway_signaling.copy()
second_layer=False
size_units = 100
design_type='bio'
design = 'signaling_gene_space_1_layer_pathway'

# # SIGNALING 2-LAYER
# df_scaler_use = df_scaled_with_pathway_signaling.copy()
# df_bio_use = df_pathway_signaling.copy()
# second_layer=True
# size_units = 100
# design_type='bio'
# design = 'signaling_gene_space_2_layer_pathway'

In [11]:
# # METABOLIC SIGNALING 1-LAYER
# df_scaler_use = df_scaled_with_pathway_signaling.copy()
# df_bio_use = df_pathway_signaling.copy()
# second_layer=False
# size_units = 100
# design_type='bio'
# design = 'signaling_gene_space_1_layer_pathway'

# # SIGNALING 2-LAYER
# df_scaler_use = df_scaled_with_pathway_signaling.copy()
# df_bio_use = df_pathway_signaling.copy()
# second_layer=True
# size_units = 100
# design_type='bio'
# design= 'signaling_gene_space_2_layer_pathway'

In [ ]:
seed(1)
df_result = pd.DataFrame()
for i_pgroups in n_groups_:
    
    print(i_pgroups)
    splitter = LeavePGroupsOut(n_groups=i_pgroups)
    splits = list(splitter.split(X, y, groups))
    ids = np.random.choice(len(splits), 20).tolist()
    list_random_selected = [splits[i] for i in ids]
    
    design_name = design+'-LeavePGroupOut_'+str(i_pgroups)

    # ## LEVAEPGROUPSOUT CONTROL
    # for i in range(len(list_random_selected)):
    #     print('EXPERIMENT -', str(i+1) , '/', str(len(list_random_selected)))
    #     print('kept            - ', list(df_scaled[df_scaled.index.isin(list_random_selected[i][0])].groupby('cell_type').size().index))
    #     print('LeavePGroupsOut - ', list(df_scaled[df_scaled.index.isin(list_random_selected[i][1])].groupby('cell_type').size().index))

    df_all_per_cell_out = pd.DataFrame()
    for i in range(len(list_random_selected)):
        print('EXPERIMENT -- ', str(i+1), '/', str(len(list_random_selected)))
        ohe_kept = OneHotEncoder()
        X_train, y_train, X_test, y_test = [], [], [], []

        df_kept_groups = df_scaler_use[df_scaler_use.index.isin(list_random_selected[i][0])]
        if design_type == 'bio':
            df_first_hidden_layer = df_bio_use[df_bio_use.index.isin(list(df_kept_groups.columns))]
        else:
            df_first_hidden_layer = pd.DataFrame()

        X_kept = df_kept_groups.iloc[:, :-1].values
        y_kept = df_kept_groups.iloc[:, -1:].values#y_ohe[list_random_selected[i][0]]
        y_ohe_kept = ohe_kept.fit_transform(y_kept).toarray()

        print(X_kept.shape)
        print(y_kept.shape)
        print(y_ohe_kept.shape)


        Xtrain, Xtest, ytrain, ytest = train_test_split(X_kept, y_kept
                                                        , test_size=test_size
                                                        , shuffle=True
                                                        , random_state=rand_state
                                                        , stratify=y_ohe_kept
                                                       )

        X_train.append(Xtrain)
        X_test.append(Xtest)
        y_train.append(ohe_kept.transform(ytrain).toarray())
        y_test.append(ohe_kept.transform(ytest).toarray())


        model_trained = src.proposed_NN(X=X_kept
                                        , y=y_kept
                                        , bio_layer=df_first_hidden_layer
                                        , design_type=design_type
                                        , size_units=size_units
                                        , select_optimizer='SGD'
                                        , second_layer=second_layer)
        model_trained.fit(X_train[0], y_train[0]
                  , epochs=epochs_default
                  , batch_size=batch_default
                  , verbose=1
                  , callbacks=callbacks
                  , validation_split=val_split)
        
        model_trained.save(src.define_folder(os.path.join(save_path, 'cell_out_'+str(i_pgroups))) + design+'_'+str(i)+'.h5')
        
        y_pred = model_trained.predict(X_test[0])
        df_proba = pd.DataFrame(y_pred, columns=list(pd.DataFrame(ohe_kept.categories_).iloc[0,:]))
        df_pred = pd.DataFrame(ohe_kept.inverse_transform(y_pred).reshape(1,-1)[0], columns=['prediction'])
        df_ground_truth = pd.DataFrame(ohe_kept.inverse_transform(np.array(y_test[0])).reshape(1,-1)[0], columns=['ground_truth'])
        df_nn = pd.concat([df_proba, df_pred, df_ground_truth], axis=1)
        df_nn['experiment_index'] = i+1
        df_all_per_cell_out = pd.concat([df_all_per_cell_out , df_nn], axis=0)
    

    list_homo, list_comp, list_vmes, list_ari, list_ami, list_fm, list_acc = [],[],[],[],[],[],[]

    for i_exp in df_all_per_cell_out['experiment_index'].unique():
        df_temp = df_all_per_cell_out[(df_all_per_cell_out['experiment_index']==i_exp)]

        list_homo.append([ design_name, homogeneity_score(df_temp['ground_truth'], df_temp['prediction']), i_exp, 'homogeneity'])
        list_comp.append([ design_name, completeness_score(df_temp['ground_truth'], df_temp['prediction']), i_exp, 'completeness'])
        list_vmes.append([ design_name, v_measure_score(df_temp['ground_truth'], df_temp['prediction']), i_exp, 'v_measure'])
        list_ari.append([ design_name, adjusted_rand_score(df_temp['ground_truth'], df_temp['prediction']), i_exp, 'ari'])
        list_ami.append([ design_name, adjusted_mutual_info_score(df_temp['ground_truth'], df_temp['prediction']), i_exp, 'ami'])
        list_fm.append([ design_name, fowlkes_mallows_score(df_temp['ground_truth'], df_temp['prediction']), i_exp, 'fowlkes_mallows'])

    result = [element for lis in [list_homo, list_comp, list_vmes, list_ari, list_ami, list_fm] for element in lis]

    df_metric = pd.DataFrame(result, columns=['experiment','score','expr','metric'])


    df_result = pd.concat([df_result 
                           , pd.DataFrame(df_metric.groupby(['experiment', 'metric']).mean()['score']).pivot_table(index='experiment', columns='metric')])
    
df_result.to_csv(os.path.join(save_path, design+'.csv'))

2
EXPERIMENT --  1 / 20
(345, 1645)
(345, 1)
(345, 14)
  -> Network designed with prior biological knowledge with 92 nodes in first hidden layer.
    -> do not need to define size_units!!
------------- NETWORK DESIGN - ARGUMENTS -------------
-- X.shape                  , (345, 1645)
-- y.shape                  , (345, 1)
-- bio_layer.shape          , (1645, 92)
-- design_type              , bio
------------- NETWORK DESIGN - CALCULATED -------------
-- input_size               , 1645
-- first_hidden_layer_size  , 92
-- size_output_layer        , 14


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')
first_hidden_layer_size 92
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer1 (Dense)    

In [38]:
df_result

score            \
metric                                                  ami       ari   
experiment                                                              
full_gene_space_1_layer_dense100-LeavePGroupOut_2  0.658438  0.413936   
full_gene_space_1_layer_dense100-LeavePGroupOut_4  0.712690  0.525283   
full_gene_space_1_layer_dense100-LeavePGroupOut_6  0.808217  0.681954   
full_gene_space_1_layer_dense100-LeavePGroupOut_8  0.869334  0.819447   

                                                                \
metric                                            completeness   
experiment                                                       
full_gene_space_1_layer_dense100-LeavePGroupOut_2     0.910127   
full_gene_space_1_layer_dense100-LeavePGroupOut_4     0.915638   
full_gene_space_1_layer_dense100-LeavePGroupOut_6     0.955766   
full_gene_space_1_layer_dense100-LeavePGroupOut_8     0.956440   

                                                                               \
metric                                            fowlkes_mallows homogeneity   
experiment                                                                      
full_gene_space_1_layer_dense100-LeavePGroupOut_2        0.559379    0.609292   
full_gene_space_1_layer_dense100-LeavePGroupOut_4        0.645923    0.676751   
full_gene_space_1_layer_dense100-LeavePGroupOut_6        0.769931    0.770296   
full_gene_space_1_layer_dense100-LeavePGroupOut_8        0.870185    0.855886   

                                                             
metric                                            v_measure  
experiment                                                   
full_gene_space_1_layer_dense100-LeavePGroupOut_2  0.721559  
full_gene_space_1_layer_dense100-LeavePGroupOut_4  0.765811  
full_gene_space_1_layer_dense100-LeavePGroupOut_6  0.840157  
full_gene_space_1_layer_dense100-LeavePGroupOut_8  0.888651

In [49]:
df_result

score            \
metric                                                  ami       ari   
experiment                                                              
full_gene_space_1_layer_dense796-LeavePGroupOut_2  0.813001  0.694269   
full_gene_space_1_layer_dense796-LeavePGroupOut_4  0.866176  0.785385   
full_gene_space_1_layer_dense796-LeavePGroupOut_6  0.882608  0.824382   
full_gene_space_1_layer_dense796-LeavePGroupOut_8  0.898635  0.859232   

                                                                \
metric                                            completeness   
experiment                                                       
full_gene_space_1_layer_dense796-LeavePGroupOut_2     0.910370   
full_gene_space_1_layer_dense796-LeavePGroupOut_4     0.927381   
full_gene_space_1_layer_dense796-LeavePGroupOut_6     0.940795   
full_gene_space_1_layer_dense796-LeavePGroupOut_8     0.949904   

                                                                               \
metric                                            fowlkes_mallows homogeneity   
experiment                                                                      
full_gene_space_1_layer_dense796-LeavePGroupOut_2        0.746307    0.812063   
full_gene_space_1_layer_dense796-LeavePGroupOut_4        0.817880    0.872150   
full_gene_space_1_layer_dense796-LeavePGroupOut_6        0.857121    0.878342   
full_gene_space_1_layer_dense796-LeavePGroupOut_8        0.891695    0.890399   

                                                             
metric                                            v_measure  
experiment                                                   
full_gene_space_1_layer_dense796-LeavePGroupOut_2  0.856387  
full_gene_space_1_layer_dense796-LeavePGroupOut_4  0.898291  
full_gene_space_1_layer_dense796-LeavePGroupOut_6  0.907293  
full_gene_space_1_layer_dense796-LeavePGroupOut_8  0.917659

In [54]:
df_result

score            \
metric                                                   ami       ari   
experiment                                                               
signaling_gene_space_1_layer_dense100-LeavePGro...  0.921370  0.887915   
signaling_gene_space_1_layer_dense100-LeavePGro...  0.941642  0.921112   
signaling_gene_space_1_layer_dense100-LeavePGro...  0.939340  0.926784   
signaling_gene_space_1_layer_dense100-LeavePGro...  0.947459  0.933704   

                                                                 \
metric                                             completeness   
experiment                                                        
signaling_gene_space_1_layer_dense100-LeavePGro...     0.945083   
signaling_gene_space_1_layer_dense100-LeavePGro...     0.958116   
signaling_gene_space_1_layer_dense100-LeavePGro...     0.957763   
signaling_gene_space_1_layer_dense100-LeavePGro...     0.960569   

                                                                    \
metric                                             fowlkes_mallows   
experiment                                                           
signaling_gene_space_1_layer_dense100-LeavePGro...        0.899515   
signaling_gene_space_1_layer_dense100-LeavePGro...        0.930333   
signaling_gene_space_1_layer_dense100-LeavePGro...        0.937325   
signaling_gene_space_1_layer_dense100-LeavePGro...        0.945697   

                                                                          
metric                                             homogeneity v_measure  
experiment                                                                
signaling_gene_space_1_layer_dense100-LeavePGro...    0.940444  0.942751  
signaling_gene_space_1_layer_dense100-LeavePGro...    0.955514  0.956809  
signaling_gene_space_1_layer_dense100-LeavePGro...    0.949929  0.953818  
signaling_gene_space_1_layer_dense100-LeavePGro...    0.956628  0.958543

In [57]:
df_result

score            \
metric                                                   ami       ari   
experiment                                                               
signaling_gene_space_1_layer_dense796-LeavePGro...  0.926862  0.896232   
signaling_gene_space_1_layer_dense796-LeavePGro...  0.948602  0.930938   
signaling_gene_space_1_layer_dense796-LeavePGro...  0.938560  0.921600   
signaling_gene_space_1_layer_dense796-LeavePGro...  0.952476  0.942172   

                                                                 \
metric                                             completeness   
experiment                                                        
signaling_gene_space_1_layer_dense796-LeavePGro...     0.948927   
signaling_gene_space_1_layer_dense796-LeavePGro...     0.962977   
signaling_gene_space_1_layer_dense796-LeavePGro...     0.956631   
signaling_gene_space_1_layer_dense796-LeavePGro...     0.963624   

                                                                    \
metric                                             fowlkes_mallows   
experiment                                                           
signaling_gene_space_1_layer_dense796-LeavePGro...        0.906972   
signaling_gene_space_1_layer_dense796-LeavePGro...        0.938991   
signaling_gene_space_1_layer_dense796-LeavePGro...        0.932874   
signaling_gene_space_1_layer_dense796-LeavePGro...        0.952663   

                                                                          
metric                                             homogeneity v_measure  
experiment                                                                
signaling_gene_space_1_layer_dense796-LeavePGro...    0.944809  0.946857  
signaling_gene_space_1_layer_dense796-LeavePGro...    0.960820  0.961894  
signaling_gene_space_1_layer_dense796-LeavePGro...    0.949586  0.953084  
signaling_gene_space_1_layer_dense796-LeavePGro...    0.961472  0.962523

In [61]:
df_result

score            \
metric                                                   ami       ari   
experiment                                                               
metsig_gene_space_1_layer_dense100-LeavePGroupO...  0.851931  0.758044   
metsig_gene_space_1_layer_dense100-LeavePGroupO...  0.877311  0.801378   
metsig_gene_space_1_layer_dense100-LeavePGroupO...  0.906386  0.851726   
metsig_gene_space_1_layer_dense100-LeavePGroupO...  0.925108  0.901158   

                                                                 \
metric                                             completeness   
experiment                                                        
metsig_gene_space_1_layer_dense100-LeavePGroupO...     0.923843   
metsig_gene_space_1_layer_dense100-LeavePGroupO...     0.938998   
metsig_gene_space_1_layer_dense100-LeavePGroupO...     0.948038   
metsig_gene_space_1_layer_dense100-LeavePGroupO...     0.952118   

                                                                    \
metric                                             fowlkes_mallows   
experiment                                                           
metsig_gene_space_1_layer_dense100-LeavePGroupO...        0.792990   
metsig_gene_space_1_layer_dense100-LeavePGroupO...        0.835543   
metsig_gene_space_1_layer_dense100-LeavePGroupO...        0.879369   
metsig_gene_space_1_layer_dense100-LeavePGroupO...        0.921789   

                                                                          
metric                                             homogeneity v_measure  
experiment                                                                
metsig_gene_space_1_layer_dense100-LeavePGroupO...    0.857052  0.888462  
metsig_gene_space_1_layer_dense100-LeavePGroupO...    0.879554  0.906773  
metsig_gene_space_1_layer_dense100-LeavePGroupO...    0.908770  0.927174  
metsig_gene_space_1_layer_dense100-LeavePGroupO...    0.930036  0.940100

In [64]:
df_result

score            \
metric                                                   ami       ari   
experiment                                                               
signaling_gene_space_1_layer_dense796-LeavePGro...  0.931600  0.903301   
signaling_gene_space_1_layer_dense796-LeavePGro...  0.940694  0.917123   
signaling_gene_space_1_layer_dense796-LeavePGro...  0.935496  0.918566   
signaling_gene_space_1_layer_dense796-LeavePGro...  0.947512  0.937272   

                                                                 \
metric                                             completeness   
experiment                                                        
signaling_gene_space_1_layer_dense796-LeavePGro...     0.951706   
signaling_gene_space_1_layer_dense796-LeavePGro...     0.957224   
signaling_gene_space_1_layer_dense796-LeavePGro...     0.953670   
signaling_gene_space_1_layer_dense796-LeavePGro...     0.960550   

                                                                    \
metric                                             fowlkes_mallows   
experiment                                                           
signaling_gene_space_1_layer_dense796-LeavePGro...        0.913227   
signaling_gene_space_1_layer_dense796-LeavePGro...        0.926830   
signaling_gene_space_1_layer_dense796-LeavePGro...        0.930217   
signaling_gene_space_1_layer_dense796-LeavePGro...        0.948629   

                                                                          
metric                                             homogeneity v_measure  
experiment                                                                
signaling_gene_space_1_layer_dense796-LeavePGro...    0.949083  0.950391  
signaling_gene_space_1_layer_dense796-LeavePGro...    0.954902  0.956056  
signaling_gene_space_1_layer_dense796-LeavePGro...    0.948137  0.950888  
signaling_gene_space_1_layer_dense796-LeavePGro...    0.956541  0.958505